In [ ]:
import torch #Pytorch used for Model creation
import torchvision
from torchvision import datasets, transforms # 'datasets' gives us MNIST, 'transforms' allows use to turn images into tensors
import matplotlib.pyplot as plt #To View Images
import torch.nn as nn #To set up the model
import torch.nn.functional as F #To calculate loss, use different activations etc.
import torch.optim as optim #To access the adam optimizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import torch.optim as optim
from operator import add,truediv

# Halt Dense Simulation
during each model update <br>
if accuracy > threshold:<br>
> share convolutional and dense layers
<br>else:<br>
> share convolutional layers only

## Classes needed



In [ ]:
# Super-class model
'''
https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
using VGG 3 which has 73% accuracy on Cifar-10
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
'''
# Now use setup from FedMax
Allen_Net = {'Conv': [32,'M',64,64,'M',64,64,'M'],
             'MLP': [(1024,512),'ReLU',(512,10)]} # 'MLP' for user-readability, not used to generate dense layers

class Model(nn.Module):
  def __init__(self,net_setup):
    super().__init__()
    self.ConvLayers = self._make_conv(net_setup['Conv'])
    self.MLP = nn.Sequential(nn.Linear(1024,512),nn.ReLU(True),nn.Linear(512,10))

    self.Conv1 = nn.Conv2d(3,32,(3,3))
    self.Conv2 = nn.Conv2d(32,32,(3,3))
    self.maxpool1 = nn.MaxPool2d((2,2))
    self.Conv3 = nn.Conv2d(32,64,(3,3))
    self.Conv4 = nn.Conv2d(64,64,(3,3))
    self.maxpool2 = nn.MaxPool2d((2,2))
    self.Conv5 = nn.Conv2d(64,128,(3,3))
    self.Conv6 = nn.Conv2d(128,128,(3,3))
    #self.maxpool3 = nn.MaxPool2d((2,2)) No "same" padding in torch, so size decreases
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(128,128)
    self.fc2 = nn.Linear(128,10)
    self.model = nn.Sequential(self.Conv1,nn.ReLU(),
                               self.Conv2,nn.ReLU(),
                               self.maxpool1,
                               self.Conv3,nn.ReLU(),
                               self.Conv4,nn.ReLU(),
                               self.maxpool2,
                               self.Conv5,nn.ReLU(),
                               self.Conv6,nn.ReLU(),
                               self.flat,
                               self.fc1,nn.ReLU(),
                               self.fc2,nn.Softmax())
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
  def _make_conv(self,layer_list):
    layer = [] # add layers based on layer_list
    in_channel = 3 # starting in_channel
    for x in layer_list:
      if x == 'M':# Add maxpool2d
        layer += [nn.MaxPool2d(kernel = 2,stride = 2)]
      else: # add conv layer
        layer += [nn.Conv2d(in_channel,x,kernel_size = 3,padding=1)],
                  nn.BatchNorm2d(x),
                  ReLU(inplace=True)]
        in_channel = x # set new channel size
      return nn.Sequential(*layer) # make layers into sequential object

  def forward(self,x):
    out1 = self.ConvLayers(x) # conv
    out2 = out1.view(out1.size(0),-1) # flatten
    out3 = self.MLP(out2) # mlp
    return F.log_softmax(out3,dim=1) # log softmax activation 
    #print("Adi")
    #return # self.model(x)
  def getConvWeights(self):
    ConvList = []
    for layer in self.ConvLayers:
      if isinstance(layer,nn.Conv2d):
        ConvList.append(layer.weight)
    #ConvList = [self.Conv1.weight,self.Conv2.weight,self.Conv3.weight,self.Conv4.weight,self.Conv5.weight,self.Conv6.weight]
    #print(self.Conv1.weight.size())
    return ConvList
  def getDenseWeights(self):
    DenseList = []
    for layer in self.MLP:
      if isinstance(layer,nn.Linear):
        DenseList.append(layer.weight)
    #DenseList = [self.fc1.weight,self.fc2.weight]
    return DenseList
  def setConvWeights(self,server):
    self.Conv1.weight.new_tensor(server.Conv1.weight)
    self.Conv2.weight.new_tensor(server.Conv2.weight)
    self.Conv3.weight.new_tensor(server.Conv3.weight)
    self.Conv4.weight.new_tensor(server.Conv4.weight)
    self.Conv5.weight.new_tensor(server.Conv5.weight)
    self.Conv6.weight.new_tensor(server.Conv6.weight)
  def setDenseWeights(self,server):
    self.fc1.weight.new_tensor(server.fc1.weight)
    self.fc2.weight.new_tensor(server.fc2.weight) 

  def set_weights(self,conv_list,dense_list):
    self.Conv1.weight.new_tensor(conv_list[0])
    self.Conv2.weight.new_tensor(conv_list[1])
    self.Conv3.weight.new_tensor(conv_list[2])
    self.Conv4.weight.new_tensor(conv_list[3])
    self.Conv5.weight.new_tensor(conv_list[4])
    self.Conv6.weight.new_tensor(conv_list[5])
    self.fc1.weight.new_tensor(dense_list[0])
    self.fc2.weight.new_tensor(dense_list[1])
  def train(self,dataloader,epochs,train=True):
    if train:
      for epoch in range(epochs):
        running_loss = 0.0
        for i,data in enumerate(dataloader):
          inputs,labels = data
          self.optimizer.zero_grad()
          outputs = self.forward(inputs)
          loss = self.criterion(outputs,labels)
          loss.backward()
          self.optimizer.step()
    #get accuracy
    correct = 0
    total = 0
    with torch.no_grad():
      for data in dataloader:
        images,labels = data
        outputs = self.forward(images)
        _,prediction = torch.max(outputs,1) # choose classification based on highest probability
        total+= labels.size(0)
        correct += (labels == prediction).sum().item()
    return float (correct/total)

In [ ]:
batch = 23
color_channels = 3
img_size = 32
x = torch.randn(batch,color_channels,img_size,img_size)
server = Model()
#server(x) # it works! May 29th 4:54pm
x = server.getConvWeights()
#x[0].size()
y = server.getDenseWeights()
#y[0].size() # should be 128x128

device1 = Model() # test of setConvWeights
x = server.getConvWeights()
print(id(x[0]))
device1.setConvWeights(server)
y = device1.getConvWeights()
print(id(y[0]))
for i,j in zip(x,y):
  assert id(x) != id(y) # shouldn't be at the same address 

device1 = Model() # test of setConvWeights
x = server.getDenseWeights()
print(id(x[0]))
device1.setDenseWeights(server)
y = device1.getDenseWeights()
print(id(y[0]))
for i,j in zip(x,y):
  assert id(x) != id(y) # shouldn't be at the same address 

print("Passed prelim tests")

140209081516192
140207624389888
140207624389168
140207624536864
Passed prelim tests


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

### Load Cifar-10 and practice training

Cifar loading code borrowed from:<br>
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html


In [ ]:
print("Load Cifar and practice training")
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
num_clients = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, # for server
                                          shuffle=True, num_workers=2)
#from : https://towardsdatascience.com/preserving-data-privacy-in-deep-learning-part-1-a04894f78029
# Dividing the training data into num_clients, with each client having equal number of images
traindata_split = torch.utils.data.random_split(trainset, [int(trainset.data.shape[0] / num_clients) for _ in range(num_clients)])
#^ for clients

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Load Cifar and practice training
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print("Set up devices for vanilla FL")
results_dict = dict()
num_devices = num_clients
for i in range(num_devices):
  results_dict["Device "+str(i)] = []
results_dict["Server"] = []
communication_rounds = 10
epoches = 1
devices = []
for i in range(num_devices):
  devices.append(Model())
server = Model()

for i in range(communication_rounds):
  print("Starting Communication Round %d"%(i))
  print("Transfer Global Model to devices")
  for device in devices:
    device.setConvWeights(server)
    device.setDenseWeights(server)

  print("Training each device")
  for i,device in enumerate(devices):
    accuracy = device.train(train_loader[i],epoches)
    results_dict["Device "+str(i)].append(accuracy)
    print("Device %d has accuracy %f"%(i,accuracy))
  
  print("Aggregate device models to server")
  global_conv = []
  global_dense = []
  for device in devices:
    global_conv.append(device.getConvWeights())
    global_dense.append(device.getDenseWeights())
  x = global_conv[0]
  y = global_dense[0]
  for idx in range(1,num_devices,1):#from index 1 to the end
    x = list(map(add,x,global_conv[idx]))
    y = list(map(add,y,global_dense[idx]))
  x = [torch.div(conv,num_devices) for conv in x]
  y = [torch.div(dense,num_devices) for dense in y]
  server.set_weights(x,y)
  accuracy = server.train(trainloader,epoches,train=False) # frick it shouldn't train!, so train = False
  print("Server Accuracy %f"%(accuracy))
  results_dict["Server"].append(accuracy)
print("Results Dict")
print(results_dict)

Set up devices for vanilla FL
Starting Communication Round 0
Transfer Global Model to devices
Training each device


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

Device 0 has accuracy 0.100000
Device 1 has accuracy 0.099440
Device 2 has accuracy 0.105040
Device 3 has accuracy 0.102800
Aggregate device models to server


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

Server Accuracy 0.100000
Starting Communication Round 1
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.100000
Device 1 has accuracy 0.104800
Device 2 has accuracy 0.105040
Device 3 has accuracy 0.102800
Aggregate device models to server
Server Accuracy 0.100000
Starting Communication Round 2
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.100000
Device 1 has accuracy 0.104800
Device 2 has accuracy 0.105040
Device 3 has accuracy 0.102800
Aggregate device models to server
Server Accuracy 0.100000
Starting Communication Round 3
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.100000
Device 1 has accuracy 0.104800
Device 2 has accuracy 0.105040
Device 3 has accuracy 0.102800
Aggregate device models to server
Server Accuracy 0.100000
Starting Communication Round 4
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.100000
Device 1 has accuracy 0.104800
Device 2 has accuracy 0.

In [ ]:
# to continue training: old needs to be changined for iid data: june 11th
for i in range(3,10):
  print("Starting Communication Round %d"%(i))
  print("Transfer Global Model to devices")
  for device in devices:
    device.setConvWeights(server)
    device.setDenseWeights(server)

  print("Training each device")
  for i,device in enumerate(devices):
    accuracy = device.train(trainloader,epoches)
    results_dict["Device "+str(i)].append(accuracy)
    print("Device %d has accuracy %f"%(i,accuracy))
  
  print("Aggregate device models to server")
  global_conv = []
  global_dense = []
  for device in devices:
    global_conv.append(device.getConvWeights())
    global_dense.append(device.getDenseWeights())
  x = global_conv[0]
  y = global_dense[0]
  for idx in range(1,num_devices,1):#from index 1 to the end
    x = list(map(add,x,global_conv[idx]))
    y = list(map(add,y,global_dense[idx]))
  x = [torch.div(conv,num_devices) for conv in x]
  y = [torch.div(dense,num_devices) for dense in y]
  server.set_weights(x,y)
  accuracy = server.train(trainloader,epoches)
  results_dict["Server"].append(accuracy)
print("Results Dict")
print(results_dict)

Starting Communication Round 3
Transfer Global Model to devices
Training each device


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

Device 0 has accuracy 0.101000
Device 1 has accuracy 0.100000
Device 2 has accuracy 0.100000
Device 3 has accuracy 0.100000
Aggregate device models to server


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

Starting Communication Round 4
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.285720
Device 1 has accuracy 0.100000
Device 2 has accuracy 0.100000
Device 3 has accuracy 0.170760
Aggregate device models to server
Starting Communication Round 5
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.340680
Device 1 has accuracy 0.232540
Device 2 has accuracy 0.100060
Device 3 has accuracy 0.321460
Aggregate device models to server
Starting Communication Round 6
Transfer Global Model to devices
Training each device
Device 0 has accuracy 0.374020
Device 1 has accuracy 0.306640
Device 2 has accuracy 0.234840


In [ ]:
print("Results Dict")
print(results_dict)

In [ ]:
# check aggregation
global_conv = []
global_dense = []
for device in devices:
  global_conv.append(device.getConvWeights())
  global_dense.append(device.getDenseWeights())
#print(global_conv)
x = global_conv[0]
y = global_dense[0]
#print(x[0][0])
for idx in range(1,num_devices,1):
  print(idx)
  x = list(map(add,x,global_conv[idx]))
  y = list(map(add,y,global_dense[idx]))
print(x[0][0])
adi = input("continue")
x = [torch.div(conv,num_devices) for conv in x]
y = [torch.div(dense,num_devices) for dense in y]
print(x[0][0])
# it works

In [ ]:
print(global_conv[0][0][0])
print(global_conv[1][0][0])
print(global_conv[2][0][0])

In [ ]:
len(global_conv)
print(range(2,num_devices,1))


In [ ]:
server.train(trainloader,epoches)

In [ ]:
a = torch.randn(4, 4)
b = torch.randn(4, 4)
torch.mean(Tensor(a,b))